In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd
import uuid
import torch

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Step 1: Define System and Simulation Parameters
N = 64  # Number of BS antennas
K = 4   # Number of users
M = 4   # Number of RF chains
omega = 0.3  # Tradeoff weight

I_max = 120  # Maximum outer iterations
sigma_n2 = 1.0  # Noise variance
mu = 0.01  # Step size for analog precoder
lambda_ = 0.01  # Step size for digital precoder
L = 20  # Number of paths for channel

# Convert to tensors
omega_t = torch.tensor(omega, dtype=torch.float32, device=device)
sigma_n2_t = torch.tensor(sigma_n2, dtype=torch.float32, device=device)
mu_t = torch.tensor(mu, dtype=torch.float32, device=device)
lambda_t = torch.tensor(lambda_, dtype=torch.float32, device=device)

# Step 2: Define Sensing Parameters
P = 3  # Number of desired sensing angles
theta_d = np.array([-60, 0, 60]) * np.pi / 180  # Desired angles in radians
delta_theta = 5 * np.pi / 180  # Half beamwidth
theta_grid = np.linspace(-np.pi / 2, np.pi / 2, 181)  # Angular grid [-90, 90] degrees
B_d = np.zeros(len(theta_grid))  # Desired beampattern
for t, theta_t in enumerate(theta_grid):
    for theta_p in theta_d:
        if abs(theta_t - theta_p) <= delta_theta:
            B_d[t] = 1

# Convert to tensors
theta_d_t = torch.tensor(theta_d, dtype=torch.float32, device=device)
theta_grid_t = torch.tensor(theta_grid, dtype=torch.float32, device=device)
B_d_t = torch.tensor(B_d, dtype=torch.float32, device=device)

# Wavenumber and antenna spacing
lambda_wave = 1  # Wavelength (normalized)
k = 2 * np.pi / lambda_wave
d = lambda_wave / 2  # Antenna spacing

k_t = torch.tensor(k, dtype=torch.float32, device=device)
d_t = torch.tensor(d, dtype=torch.float32, device=device)

import h5py
SNR_dB_array = np.arange(0, 12.1, 0.1)
# Load Psi data (from MATLAB .mat file)
with h5py.File('Psi_all.mat', 'r') as f:
    Psi_h5 = f['Psi_all']

    # If stored as MATLAB complex structure (real/imag parts separate)
    if np.issubdtype(Psi_h5.dtype, np.void):
        real = Psi_h5['real'][()]
        imag = Psi_h5['imag'][()]
        Psi_all = real + 1j * imag
    else:
        Psi_all = np.array(Psi_h5)

# Ensure Psi_all has shape: (num_SNRs, M, N)
Psi_all = np.squeeze(Psi_all)  # remove singleton dimensions if any


def compute_psi(snr_db):
    """
    Selects the Psi matrix corresponding to the closest SNR value.
    """
    # Find index of closest SNR
    idx = np.argmin(np.abs(SNR_dB_array - snr_db))

    # Select corresponding Psi  
    Psi = Psi_all[idx, :, :]

    return Psi

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

# ==== Config ====
num_epochs = 30
num_channels = 500

from lib.dnn import UPGANet, upganet_loss
from lib.support_functions_torch import (
    generate_channel_torch_batch,
    proposed_initialization_torch_batch_multiSNR
)

# print(f"Training config: epochs={num_epochs}, channels/epoch={num_channels}, SNR={snr_max} dB")


# snr_db = snr_max
snr_min, snr_max, snr_step = 12, 12, 2
snr_range = torch.arange(snr_min, snr_max + snr_step, snr_step)


P_BS_list = []

for snr_db in snr_range:
    P_BS = sigma_n2 * 10**(snr_db / 10)
    P_BS_t = torch.tensor(P_BS, dtype=torch.float32, device=device)

    P_BS_list.append(P_BS_t)

# # ==== Pre-generate channel batch ====
# H_batch = generate_channel_batch(N, M, L=20, batch_size=num_channels, device=device)
# print(f"Channel batch shape: {H_batch.shape}")

# ==== Training ====
J_values = [10]

for J in J_values:
    print(f"\nTraining UPGANet (J={J})")
    model = UPGANet(N, M, K, omega, I_max=I_max, J=J).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)#for J= 1
    decay_rate = 0.97
    # optimizer = optim.Adam(model.parameters(), lr=0.001) #for J= 10
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay_rate)


    loss_history, mu_history, lambda_history = [], [], []

    
    for epoch in range(num_epochs):
        H_batch = generate_channel_torch_batch(N, M, L=20, batch_size=num_channels, device=device)
        A0, D0, P_BS_used = proposed_initialization_torch_batch_multiSNR(H_batch, theta_d_t, N, M, K, P_BS_list, device=device)
        total_loss = 0.0
        for ch_idx in range(num_channels):
            H_ch = H_batch[ch_idx]
            A0_ch = A0[ch_idx]
            D0_ch = D0[ch_idx]
            P_BS_t = P_BS_used[ch_idx]

            Psi = compute_psi(P_BS_t.cpu().item() / sigma_n2 * 10)  # Convert back to SNR in dB
            Psi = torch.tensor(Psi, dtype=torch.complex64, device=device)
            
            # A0, D0 = proposed_initialization_torch(H_ch, theta_d_t, N, M, K, P_BS_t, device=device)
            A_final, D_final = model(H_ch, A0_ch, D0_ch, Psi, sigma_n2_t, P_BS_t)

            loss = upganet_loss(H_ch, A_final, D_final, Psi, sigma_n2_t, omega)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / num_channels
        loss_history.append(avg_loss)

        with torch.no_grad():
            mu_mean = model.layers[0].mu.mean().item()
            lambda_val = model.layers[0].lambda_.item()
        mu_history.append(mu_mean)
        lambda_history.append(lambda_val)
        scheduler.step()
        current_lr = scheduler.get_last_lr()[0]

        print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {avg_loss:.6f} | μ={mu_mean:.6f}, λ={lambda_val:.6f}")

    # ==== Save model ====
    save_path = f"upganet_J{J}_final.pth"
    torch.save(model.state_dict(), save_path)
    print(f"Model saved: {save_path}")

    # ==== Plot results ====
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    axes[0].plot(loss_history, linewidth=2)
    axes[0].set_title("Training Loss")
    axes[0].set_xlabel("Epoch")
    axes[0].set_ylabel("Loss")
    axes[0].grid(True, alpha=0.3)

    axes[1].plot(mu_history, linewidth=2, color="orange")
    axes[1].set_title("Learned Step Size μ")
    axes[1].set_xlabel("Epoch")
    axes[1].grid(True, alpha=0.3)

    axes[2].plot(lambda_history, linewidth=2, color="green")
    axes[2].set_title("Learned Step Size λ")
    axes[2].set_xlabel("Epoch")
    axes[2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

print("\nTraining complete ✅")


/tmp/ipykernel_3893321/467556693.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  P_BS_t = torch.tensor(P_BS, dtype=torch.float32, device=device)



Training UPGANet (J=10)
Epoch [1/30] | Loss: 55.829961 | μ=0.021512, λ=-0.009596
Epoch [2/30] | Loss: 56.522087 | μ=0.042546, λ=0.016392
Epoch [3/30] | Loss: 54.926397 | μ=0.070009, λ=0.025293


In [ ]:
import torch
from lib.dnn import UPGANet
from lib.support_functions_torch import (
    compute_rate_torch,
    generate_channel_torch_batch,
    proposed_initialization_torch,
    compute_psi
)

def evaluate_untrained_upganet(mu_val=0.01, lambda_val=0.01, J=1, num_realizations=100, snr_range=range(0, 13, 2)):
    """
    Evaluate an untrained UPGANet model with fixed μ and λ values
    to compute the average achievable rate vs SNR.
    """
    print("="*70)
    print(f"EVALUATING UNTRAINED UPGANet (μ={mu_val}, λ={lambda_val}, J={J})")
    print("="*70)
    print(f"Device: {device}\n")

    # Instantiate untrained model
    untrained_model = UPGANet(N, M, K, omega, I_max=I_max, J=J).to(device)

    # Set μ and λ manually
    with torch.no_grad():
        for layer in untrained_model.layers:
            layer.mu.data.fill_(mu_val)
            layer.lambda_.data.fill_(lambda_val)

    untrained_model.eval()

    R_untrained_list = []

    for snr_db in snr_range:
        R_sum = 0.0
        P_BS = sigma_n2 * 10**(snr_db / 10)
        P_BS_t = torch.tensor(P_BS, dtype=torch.float32, device=device)
        Psi_t = torch.tensor(compute_psi(snr_db), dtype=torch.cfloat, device=device)
        H_batch = generate_channel_torch_batch(N, M, L=20, batch_size=num_realizations, device=device)

        for realization in range(num_realizations):
            H_t = H_batch[realization]

            # Initialization (same as in training)
            A0, D0 = proposed_initialization_torch(H_t, theta_d_t, N, M, K, P_BS_t, device=device)

            with torch.no_grad():
                A_final, D_final = untrained_model(H_t, A0, D0, Psi_t, sigma_n2_t, P_BS_t)

            # Compute achievable rate
            R = compute_rate_torch(H_t, A_final, D_final, sigma_n2_t)
            R_sum += R.item()

        R_avg = R_sum / num_realizations
        R_untrained_list.append(R_avg)
        print(f"SNR = {snr_db:2d} dB → Avg R = {R_avg:.4f} bps/Hz")

    return R_untrained_list


# Evaluate untrained (μ=λ=0.01)
untrained_results = evaluate_untrained_upganet(
    mu_val=-0.176042, lambda_val=0.021704, J=1,
    num_realizations=100, snr_range=range(0, 13, 2)
)

# ==== Plot comparison ====
import matplotlib.pyplot as plt

snr_vals = list(range(0, 13, 2))

plt.figure(figsize=(6,4))
plt.plot(snr_vals, untrained_results, 'o--', label='Untrained (μ=λ=0.01)')
# plt.plot(snr_vals, trained_results, 's-', label='Trained UPGANet')
plt.xlabel('SNR (dB)')
plt.ylabel('Average Achievable Rate (bps/Hz)')
plt.title('UPGANet Performance: Before vs After Training')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


EVALUATING UNTRAINED UPGANet (μ=-0.176042, λ=0.021704, J=1)
Device: cpu

SNR =  0 dB → Avg R = 2.3815 bps/Hz
SNR =  2 dB → Avg R = 2.4429 bps/Hz
SNR =  4 dB → Avg R = 2.7596 bps/Hz
SNR =  6 dB → Avg R = 2.7351 bps/Hz


KeyboardInterrupt: 